In [ ]:
import tensorflow as tf
from keras.applications import ResNet50
from keras.models import Model
from keras import models
from keras import layers
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import splitfolders
from keras.preprocessing.image import ImageDataGenerator

# Eingabe- und Ausgabeordner definieren
input_folder = './ASL_unique'
output_folder = './sorted_ResNet50uniqueOpti'

# Daten in Trainings- und Validierungsordner aufteilen
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.2))

# Zielgröße der Bilder definieren
target_size = (224, 224)

# Datenagmentierung für Trainings-, Validierungs- und Testdaten erstellen
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Daten-Generatoren für Trainings- und Validierungsdaten erstellen
train_generator = train_datagen.flow_from_directory(
    output_folder + '/train',
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    output_folder + '/val',
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
# Eingabeordner definieren
input_folder = './ASL_test'

# Daten-Generator für Testdaten erstellen
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    input_folder,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))

# Schritt 2: Fügen Sie dem ResNet-50-Modell einen benutzerdefinierten Klassifikator hinzu
x = layers.Flatten()(resnet_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
output = layers.Dense(8, activation='softmax')(x)  

model_resnet = models.Model(inputs=resnet_model.input, outputs=output)

# Schritt 3: Kompilieren Sie das Modell
model_resnet.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Schritt 4: Frühzeitiges Stoppen einrichten
early_stop_resnet = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Schritt 5: Trainieren Sie das Modell und speichern Sie die Historie
history_resnet = model_resnet.fit(train_generator, epochs=25, validation_data=val_generator, callbacks=[early_stop_resnet])

# Schritt 6: Model Summary anzeigen
model_resnet.summary()

# Schritt 7: Testen Sie das Modell und bewerten Sie es auf dem Testset
loss_resnet, accuracy_resnet = model_resnet.evaluate(test_generator)
print(f'Test loss (ResNet-50): {loss_resnet}')
print(f'Test accuracy (ResNet-50): {accuracy_resnet}')

# Schritt 8: Plot Loss und Accuracy über die Zeit
plt.figure(figsize=(12, 6))

# Plot Loss über die Zeit
plt.subplot(1, 2, 1)
plt.plot(history_resnet.history['loss'], label='Training Loss')
plt.plot(history_resnet.history['val_loss'], label='Validation Loss')
plt.title('Loss over Time')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy über die Zeit
plt.subplot(1, 2, 2)
plt.plot(history_resnet.history['accuracy'], label='Training Accuracy')
plt.plot(history_resnet.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Time')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Verzeichnis zum Speichern der Bilder
save_directory = './Result'
os.makedirs(save_directory, exist_ok=True)

# Iteration über die Testdaten
for _ in range(1):
    # Vorhersagen des Modells für die Testdaten
    predictions = model_resnet.predict(test_generator)
    
    # Wahre Labels der Testdaten
    true_labels = test_generator.labels
    
    # Indizes der inkorrekten Vorhersagen finden
    incorrect_indices = np.where(np.argmax(predictions, axis=1) != true_labels)[0]

    # Iteration über die inkorrekten Vorhersagen
    for i in range(len(incorrect_indices)):
        # Index des Batches und Index innerhalb des Batches
        batch_index = incorrect_indices[i] // test_generator.batch_size
        in_batch_index = incorrect_indices[i] % test_generator.batch_size

        # Bild und zugehörige Labels extrahieren
        image = test_generator[batch_index][0][in_batch_index]
        true_label = true_labels[incorrect_indices[i]]
        predicted_label = np.argmax(predictions[incorrect_indices[i]])

        # Bild anzeigen
        plt.figure(figsize=(8, 8))
        plt.imshow(image)
        plt.title(f'Wahres Label: {true_label}, Vorhergesagtes Label: {predicted_label}')

        # Bild speichern
        image_filename = f'falsches_bild_{i+1}_wahr_{true_label}_vorhergesagt_{predicted_label}.png'
        image_path = os.path.join(save_directory, image_filename)
        plt.savefig(image_path)

        # Plot schließen, um nicht angezeigt zu werden
        plt.close()

In [ ]:
print("Class indices in test_generator:", test_generator.class_indices)